# 线程进程背景知识
现代操作系统比如Mac OS X，UNIX，Linux，Windows等，都是支持“多任务”的操作系统。

什么叫“多任务”呢？简单地说，就是操作系统可以同时运行多个任务。打个比方，你一边在用浏览器上网，一边在听MP3，一边在用Word赶作业，这就是多任务，至少同时有3个任务正在运行。还有很多任务悄悄地在后台同时运行着，只是桌面上没有显示而已。

现在，多核CPU已经非常普及了，但是，即使过去的单核CPU，也可以执行多任务。由于CPU执行代码都是顺序执行的，那么，单核CPU是怎么执行多任务的呢？

答案就是操作系统轮流让各个任务交替执行，任务1执行0.01秒，切换到任务2，任务2执行0.01秒，再切换到任务3，执行0.01秒……这样反复执行下去。表面上看，每个任务都是交替执行的，但是，由于CPU的执行速度实在是太快了，我们感觉就像所有任务都在同时执行一样。

真正的并行执行多任务只能在多核CPU上实现，但是，由于任务数量远远多于CPU的核心数量，所以，操作系统也会自动把很多任务轮流调度到每个核心上执行。

对于操作系统来说，一个任务就是一个进程（Process），比如打开一个浏览器就是启动一个浏览器进程，打开一个记事本就启动了一个记事本进程，打开两个记事本就启动了两个记事本进程，打开一个Word就启动了一个Word进程。

有些进程还不止同时干一件事，比如Word，它可以同时进行打字、拼写检查、打印等事情。在一个进程内部，要同时干多件事，就需要同时运行多个“子任务”，我们把进程内的这些“子任务”称为线程（Thread）。

由于每个进程至少要干一件事，所以，一个进程至少有一个线程。当然，像Word这种复杂的进程可以有多个线程，多个线程可以同时执行，多线程的执行方式和多进程是一样的，也是由操作系统在多个线程之间快速切换，让每个线程都短暂地交替运行，看起来就像同时执行一样。当然，真正地同时执行多线程需要多核CPU才可能实现。

我们前面编写的所有的Python程序，都是执行单任务的进程，也就是只有一个线程。如果我们要同时执行多个任务怎么办？

有两种解决方案：

一种是启动多个进程，每个进程虽然只有一个线程，但多个进程可以一块执行多个任务。

还有一种方法是启动一个进程，在一个进程内启动多个线程，这样，多个线程也可以一块执行多个任务。

当然还有第三种方法，就是启动多个进程，每个进程再启动多个线程，这样同时执行的任务就更多了，当然这种模型更复杂，实际很少采用。

总结一下就是，多任务的实现有3种方式：

- 多进程模式；
- 多线程模式；
- 多进程+多线程模式。

同时执行多个任务通常各个任务之间并不是没有关联的，而是需要相互通信和协调，有时，任务1必须暂停等待任务2完成后才能继续执行，有时，任务3和任务4又不能同时执行，所以，多进程和多线程的程序的复杂度要远远高于我们前面写的单进程单线程的程序。

因为复杂度高，调试困难，所以，不是迫不得已，我们也不想编写多任务。但是，有很多时候，没有多任务还真不行。想想在电脑上看电影，就必须由一个线程播放视频，另一个线程播放音频，否则，单线程实现的话就只能先把视频播放完再播放音频，或者先把音频播放完再播放视频，这显然是不行的。

Python既支持多进程，又支持多线程，我们会讨论如何编写这两种多任务程序。

## 小结

线程是最小的执行单元，而进程由至少一个线程组成。如何调度进程和线程，完全由操作系统决定，程序自己不能决定什么时候执行，执行多长时间。
多进程和多线程的程序涉及到同步、数据共享的问题，编写起来更复杂。


# 什么是线程什么是进程
几乎所有的操作系统都支持进程的概念，所有运行中的任务通常对应一个进程（Process）。当一个程序进入内存运行时，即变成一个进程。进程是处于运行过程中的程序，并且具有一定的独立功能。进程是系统进行资源分配和调度的一个独立单位。
一般而言，进程包含如下三个特征：
- 独立性：进程是系统中独立存在的实体，它可以拥有自己的独立的资源，每一个进程都拥有自己的私有的地址空间。在没有经过进程本身允许的情况下，一个用户进程不可以直接访问其他进程的地址空间。
- 动态性：进程与程序的区别在于，程序只是一个静态的指令集合，而进程是一个正在系统中活动的指令集合。在进程中加入了时间的概念。进程具有自己的生命周期和各种不同的状态，在程序中是没有这些概念的。
- 并发性：多个进程可以在单个处理器上并发执行，多个进程之间不会互相影响。

>并发（Concurrency）和并行（Parallel）是两个概念，并行指在同一时刻有多条指令在多个处理器上同时执行；并发才旨在同一时刻只能有一条指令执行，但多个进程指令被快速轮换执行，使得在宏观上具有多个进程同时执行的效果。

大部分操作系统都支持多进程并发执行，现代的操作系统几乎都支持同时执行多个任务。例如，程序员一边开着开发工具在写程序，一边开着参考手册备查，同时还使用电脑播放音乐……除此之外，每台电脑运行时还有大量底层的支撑性程序在运行……这些进程看上去像是在同时工作。但事实的真相是，对于一个 CPU 而言，在某个时间点它只能执行一个程序。也就是说，只能运行一个进程，CPU 不断地在这些进程之间轮换执行。

多线程则扩展了多进程的概念，使得同一个进程可以同时并发处理多个任务。线程（Thread）也被称作轻量级进程（Lightweight Process），线程是进程的执行单元。就像进程在操作系统中的地位一样，线程在程序中是独立的、并发的执行流。

当进程被初始化后，主线程就被创建了。对于绝大多数的应用程序来说，通常仅要求有一个主线程，但也可以在进程内创建多个顺序执行流，这些顺序执行流就是线程，每一个线程都是独立的。

线程是进程的组成部分，一个进程可以拥有多个线程，一个线程必须有一个父进程。线程可以拥有自己的堆栈、自己的程序计数器和自己的局部变量，但不拥有系统资源，它与父进程的其他线程共享该进程所拥有的全部资源。因为多个线程共享父进程里的全部资源，因此编程更加方便；但必须更加小心，因为需要确保线程不会妨碍同一进程中的其他线程。

线程可以完成一定的任务，可以与其他线程共享父进程中的共享变量及部分环境，相互之间协同未完成进程所要完成的任务。

线程是独立运行的，它并不知道进程中是否还有其他线程存在。线程的运行是抢占式的，也就是说，当前运行的线程在任何时候都可能被挂起，以便另外一个线程可以运行。

一个线程可以创建和撤销另一个线程，同一个进程中的多个线程之间可以并发运行。

从逻辑的角度来看，多线程存在于一个应用程序中，让一个应用程序可以有多个执行部分同时执行，但操作系统无须将多个线程看作多个独立的应用，对多线程实现调度和管理，以及资源分配。线程的调度和管理由进程本身负责完成。

简而言之，一个程序运行后至少有一个进程，在一个进程中可以包含多个线程，但至少要包含一个主线程。

**归纳起来可以这样说，操作系统可以同时执行多个任务，每一个任务就是一个进程，进程可以同时执行多个任务，每一个任务就是一个线程。**

## 多线程的好处
线程在程序中是独立的、并发的执行流。与分隔的进程相比，进程中线程之间的隔离程度要小，它们共享内存、文件句柄和其他进程应有的状态。

因为线程的划分尺度小于进程，使得多线程程序的并发性高。进程在执行过程中拥有独立的内存单元，而多个线程共享内存，从而极大地提高了程序的运行效率。

线程比进程具有更高的性能，这是由于同一个进程中的线程都有共性多个线程共享同一个进程的虚拟空间。线程共享的环境包括进程代码段、进程的公有数据等，利用这些共享的数据，线程之间很容易实现通信。

操作系统在创建进程时，必须为该进程分配独立的内存空间，并分配大量的相关资源，但创建线程则简单得多。因此，使用多线程来实现并发比使用多进程的性能要高得多。

总结起来，使用多线程编程具有如下几个优点：

- 进程之间不能共享内存，但线程之间共享内存非常容易。
- 操作系统在创建进程时，需要为该进程重新分配系统资源，但创建线程的代价则小得多。因此，使用多线程来实现多任务并发执行比使用多进程的效率高。
- Python 语言内置了多线程功能支持，而不是单纯地作为底层操作系统的调度方式，从而简化了 Python 的多线程编程。


# 并发编程

？广义的并发编程是指使程序能够在一段时间内处理多个任务的一种编程方法，可通多线程和多进程来实现。

并发编程话题下包含多线程，多线程包含并行编程
![](./res/Lesson8_1.jpg)

- 进程：
一个程序的执行过程或者一个任务。

- 并发：
是指单位时间内,可以处理事情的能力[特别注意这个单位时间] 。
是伪并行，看起来是同时运行，其实通过单个cpu+多道技术就可以实现并发。
在操作系统中，是指一个时间段中有几个程序都处于已启动运行到运行完毕之间，且这几个程序都是在同一个处理机上运行，但任一个时刻点上只有一个程序在处理机上运行。简言之，是指系统具有**处理多个任务的能力**。

- 并行：
是指同一时刻可以同时处理事情的能力。
同时运行，只有具备多个cpu或多核cpu才能实现并行。
当系统有一个以上CPU时,则线程的操作有可能非并发。当一个CPU执行一个线程时，另一个CPU可以执行另一个线程，两个线程互不抢占CPU资源，可以同时进行，这种方式我们称之为并行(Parallel)。简言之，是指系统具有**同时处理多个任务的能力**。

并发：Concurrency

两个队交替着去一个咖啡机买咖啡

并行：Parallelism

两个队分别取两个咖啡机买咖啡
- ![](./res/Lesson8_2.jpg)

## Python中并发编程相关模块

threading 模块：是Python内置的多线程类。（基于线程的并行编程）

线程也叫轻量级进程，它是一个基本的cpu执行单元，也是程序执行过程中的最小单元

multiprocessing模块：是Python内置的多进程类。（基于进程的并行编程）

进程就是一个程序在一个数据集上的一次动态执行过程。进程一般由程序，数据集，进程控制块三部分组成。

线程与进程的关系：
- 线程是属于进程的
- 线程运行在进程空间内

### 创建线程

#### Python的默认主线程创建：

当进程被初始化后，主线程就被创建了。python执行代码默认通常仅创建一个主线程。
>调用threading模块current_thread()函数获取当前线程,线程对象的getName()方法获取当前线程的名字
>`threading.current_thread().getName()` / `threading.current_thread().name`

_____
```python
import time, threading

def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
loop()
print('thread %s ended.' % threading.current_thread().name)
```
- 结果：

```python
thread MainThread is running...
thread MainThread is running...
thread MainThread >>> 1
thread MainThread >>> 2
thread MainThread >>> 3
thread MainThread >>> 4
thread MainThread >>> 5
thread MainThread ended.
thread MainThread ended.

```
-----

#### Python手动创建线程：
由于任何进程默认就会启动一个线程，我们把该线程称为主线程，主线程又可以启动新的线程。

```python
import time, threading
# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t1 = threading.Thread(target=loop, name='LoopThread1')
t1.start()
t2 = threading.Thread(target=loop, name='LoopThread2')
t2.start()
print('thread %s ended.' % threading.current_thread().name)
```

结果：
```python
thread MainThread is running...
thread LoopThread1 is running...
thread LoopThread1 >>> 1
thread LoopThread2 is running...
thread MainThread ended.
thread LoopThread2 >>> 1
thread LoopThread1 >>> 2
thread LoopThread2 >>> 2
thread LoopThread1 >>> 3
thread LoopThread2 >>> 3
thread LoopThread1 >>> 4
thread LoopThread2 >>> 4
thread LoopThread1 >>> 5
thread LoopThread2 >>> 5
thread LoopThread1 ended.
thread LoopThread2 ended.
```

上面的程序中的主程序包含print函数，并线程创建函数启动一个新的线程：
创建了一个 Thread 对象，该线程的 target 为 loop，这意味着它会将 loop 函数作为线程执行体。接下来程序调用 start() 方法来启动t线程。
虽然上面程序只显式创建并启动了2个线程，但实际上程序有3个线程，即程序显式创建的2个子线程和主线程。前面己经提到，当 Python 程序开始运行后，程序至少会创建一个主线程，主线程的线程执行体就是程序中的主程序（没有放在任何函数中的代码）。
>在进行多线程编程时，不要忘记 Python 程序运行时默认的主线程，主程序部分（没有放在任何函数中的代码）就是主线程的线程执行体。

程序中共包含三个线程，这三个线程的执行没有先后顺序，它们以并发方式执行：LoopThread1 执行一段时间，然后可能 LoopThread2 或 MainThread 获得 CPU 执行一段时间，接下来又换其他线程执行，这就是典型的线程并发执行，CPU 以快速轮换的方式在多个线程之间切换，从而给用户一种错觉，即多个线程似乎同时在执行。

通过上面介绍不难看出多线程的意义，如果不使用多线程，主程序直接调用两次 loop() 函数，那么程序必须等第一次调用的 loop() 函数执行完成，才会执行第二次调用的 loop() 函数；必须等第二次调用的 loop() 函数执行完成，才会继续向下执行主程序。

-----

Python 主要通过两种方式来创建线程：
- 使用 threading 模块的 Thread 类的构造器创建线程。
- 继承 threading 模块的 Thread 类创建线程类。
>- [threading文档](https://docs.python.org/zh-cn/3.7/library/threading.html)

#### 调用 Thread 类的构造器创建线程：

调用 Thread 类的构造器创建线程很简单，直接调用 threading.Thread 类的如下构造器创建线程：

`__init__(self, group=None, target=None, name=None, args=(), kwargs=None, *,daemon=None)`
上面的构造器涉及如下几个参数：
- group：指定该线程所属的线程组。目前该参数还未实现，因此它只能设为 None。
- target：指定该线程要调度的目标方法。是用于 run() 方法调用的可调用对象。
- name 是线程名称。默认情况下，由 "Thread-N" 格式构成一个唯一的名称，其中 N 是小的十进制数。
- args：指定一个元组，以位置参数的形式为 target 指定的函数传入参数。元组的第一个元素传给 target 函数的第一个参数，元组的第二个元素传给 target 函数的第二个参数……依此类推。
- kwargs：指定一个字典，以关键字参数的形式为 target 指定的函数传入参数。
- daemon：指定所构建的线程是否为后代线程。
 > run()代表线程活动的方法。你可以在子类型里重载这个方法。 标准的 run() 方法会对作为 target 参数传递给该对象构造器的可调用对象（如果存在）发起调用，并附带从 args 和 kwargs 参数分别获取的位置和关键字参数。


通过 Thread 类的构造器创建井启动多线程的步骤如下：
- 调用 Thread 类的构造器创建线程对象。在创建线程对象时，target 参数指定的函数将作为线程执行体。
- 调用线程对象的 start() 方法启动该线程。

#### 继承 Thread 类创建线程类:

通过继承 Thread 类来创建并启动线程的步骤如下：
- 定义 Thread 类的子类，并重写该类的 run() 方法。run() 方法的方法体就代表了线程需要完成的任务，因此把 run() 方法称为线程执行体。
- 创建 Thread 子类的实例，即创建线程对象。
- 调用线程对象的 start() 方法来启动线程。

#### Thread类方法：
t.start() ：激活线程。被调用后，线程被cpu调度后自动执行线程对象的run()方法

t.join():逐步执行每个线程，执行完毕后继续往下执行

#### star()/run():
启动线程使用 start() 方法，而不是 run() 方法
启动线程的正确方法是调用 Thread 对象的 start() 方法，而不是直接调用 run() 方法，否则就变成单线程程序了。
调用 start() 方法来启动线程，系统会把该 run() 方法当成线程执行体来处理；但如果直接调用线程对象的 run() 方法，则 run() 方法立即就会被执行，而且在该方法返回之前其他线程无法并发执行。也就是说，如果直接调用线程对象的 run() 方法，则系统把线程对象当成一个普通对象，而 run() 方法也是一个普通方法，而不是线程执行体。
在调用线程对象的 start() 方法之后，该线程立即进入就绪状态（相当于“等待执行”），但该线程并未真正进入运行状态。

#### Python线程的生命周期:
当线程被创建并启动以后，它既不是一启动就进入执行状态的，也不是一直处于执行状态的，在线程的生命周期中，它要经过新建（new）、就绪（Ready）、运行（Running）、阻塞（Blocked）和死亡（Dead）5 种状态。当线程被创建并启动以后，它既不是一启动就进入执行状态的，也不是一直处于执行状态的，在线程的生命周期中，它要经过新建（new）、就绪（Ready）、运行（Running）、阻塞（Blocked）和死亡（Dead）5 种状态。

尤其是当线程启动以后，它不可能一直“霸占”着 CPU 独自运行，所以 CPU 需要在多个线程之间切换，于是线程状态也会多次在运行、就绪之间转换。

当程序创建了一个 Thread 对象或 Thread 子类的对象之后，该线程就处于新建状态，和其他的 Python 对象一样，此时的线程对象并没有表现出任何线程的动态特征，程序也不会执行线程执行体。

当线程对象调用 start() 方法之后，该线程处于就绪状态，Python 解释器会为其创建方法调用栈和程序计数器，处于这种状态中的线程并没有开始运行，只是表示该线程可以运行了。至于该线程何时开始运行，取决于 Python 解释器中线程调度器的调度。

在调用线程对象的 start() 方法之后，该线程立即进入就绪状态（相当于“等待执行”），但该线程并未真正进入运行状态。

如果处于就绪状态的线程获得了 CPU，开始执行 run() 方法的线程执行体，则该线程处于运行状态。

当前正在执行的线程被阻塞之后，其他线程就可以获得执行的机会。被阻塞的线程会在合适的时候重新进入就绪状态，注意是就绪状态，而不是运行状态。也就是说，被阻塞线程的阻塞解除后，必须重新等待线程调度器再次调度它。

线程会以如下方式结束，结束后就处于死亡状态：
 - run() 方法或代表线程执行体的 target 函数执行完成，线程正常结束。
 - 线程抛出一个未捕获的 Exception 或 Error。

>注意，当主线程结束时，其他线程不受任何影响，并不会随之结束。一旦子线程启动起来后，它就拥有和主线程相同的地位，不会受主线程的影响。(在不启用守护的情况下)


### Python守护线程及作用

有一种线程，它是在后台运行的，它的任务是为其他线程提供服务，这种线程被称为“后台线程（Daemon Thread）”，又称为“守护线程”

后台线程有一个特征，如果所有的前台线程都死亡了，那么后台线程会自动死亡。

调用 Thread 对象的 daemon 属性可以将指定线程设置成后台线程。下面程序将指定线程设置成后台线程，可以看到当所有的前台线程都死亡后，后台线程随之死亡。当在整个虚拟机中只剩下后台线程时，程序就没有继续运行的必要了，所以程序也就退出了。

```python
import time, threading
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.setDaemon(True)
t.start()
#t.join()
print('thread %s ended.' % threading.current_thread().name)
```
结果：
```python
thread MainThread is running...
thread LoopThread is running...
thread MainThread ended.
thread LoopThread >>> 1
```
从上面的程序可以看出，主线程默认是前台线程，t线程默认也是前台线程,通t.setDaemon(True)设置t线程为后台线程。但并不是所有的线程默认都是前台线程，有些线程默认就是后台线程。前台线程创建的子线程默认是前台线程，后台线程创建的子线程默认是后台线程。

可见，创建后台线程有两种方式：
- 主动将线程的 daemon 属性设置为 True。
- 后台线程启动的线程默认是后台线程。

>如果要将某个线程设置为后台线程，则必须在该线程启动之前进行设置

### Thread join()用法:
Thread 提供了让一个线程等待另一个线程完成的 join() 方法。当在某个程序执行流中调用其他线程的 join() 方法时，调用线程将被阻塞，直到被 join() 方法加入的 join 线程执行完成。
join() 方法通常由使用线程的程序调用，以将大问题划分成许多小问题，并为每个小问题分配一个线程。当所有的小问题都得到处理后，再调用主线程来进一步操作。

```python
import threading

# 定义action函数准备作为线程执行体使用
def action(max):
    for i in range(max):
        print(threading.current_thread().name + " " + str(i))

# 启动子线程
threading.Thread(target=action, args=(8,), name="新线程").start()
for i in range(8):
    if i == 4:
        jt = threading.Thread(target=action, args=(8,), name="被Join的线程")
        jt.start()
        # 主线程调用了jt线程的join()方法，主线程
        # 必须等jt执行结束才会向下执行
        jt.join()
    print(threading.current_thread().name + " " + str(i))
```
结果：
```
新线程 0
MainThread 0
新线程 1
MainThread 1
新线程 2
MainThread 2
新线程 3
MainThread 3
新线程 4
被Join的线程 0
新线程 5
被Join的线程 1
新线程 6
被Join的线程 2
被Join的线程 3
新线程 7
被Join的线程 4
被Join的线程 5
被Join的线程 6
被Join的线程 7
MainThread 4
MainThread 5
MainThread 6
MainThread 7
```
上面程序中一共有三个线程，主程序开始时就启动了名为“新线程”的子线程，该子线程将会和主线程并发执行。当主线程的循环变量 i 等于4 时，启动了名为“被Join的线程”的线程，该线程不会和主线程并发执行，主线程必须等该线程执行结束后才可以向下执行。
在名为“被Join的线程”的线程执行时，实际上只有两个子线程并发执行，而主线程处于等待状态。

> `join(timeout=None)`方法可以指定一个 timeout 参数，该参数指定等待被 join 的线程的时间最长为 timeout 秒。如果在 timeout 秒内被 join 的线程还没有执行结束，则不再等待。

Thread 类的构造器创建线程：

In [ ]:
## 终端运行

# Thread 类的构造器创建线程：
import threading
import time
 
def counter(n):  
    cnt = 0;  
    for i in range(n):  
        cnt += 1
        time.sleep(0.1)
        print(cnt)
               
th = threading.Thread(target=counter, args=(10,));   
th.start();  
print('main thread task done')

继承 Thread 类创建线程类

In [ ]:
#终端运行

#继承 Thread 类创建线程类
import threading
  
def counter():  
    cnt = 0;  
    for i in range(10000):  
        for j in range(i):  
            cnt += j;  

class SubThread(threading.Thread):  
    def __init__(self, name):
        #初始化基类
        threading.Thread.__init__(self, name=name) 
        pass
  
    def run(self):  
        i = 0;  
        while i < 3:  
            print(self.name,'counting...\n') 
            counter();  
            print(self.name,'finish\n')  
            i += 1;  

th = SubThread('thread-1')
th.start()
th.join()
print('all done') 

In [ ]:
import threading

# 通过继承threading.Thread类来创建线程类
class FkThread(threading.Thread):
    def __init__(self): 
        threading.Thread.__init__(self)
        self.i = 0
    # 重写run()方法作为线程执行体
    def run(self): 
        while self.i < 100:
            # 调用threading模块current_thread()函数获取当前线程
            # 线程对象的getName()方法获取当前线程的名字
            print(threading.current_thread().getName() +  " " + str(self.i))
            self.i += 1
# 下面是主程序（也就是主线程的执行体）
for i in range(100):
    # 调用threading模块current_thread()函数获取当前线程
    print(threading.current_thread().getName() +  " " + str(i))
    if i == 20:
        # 创建并启动第一个线程
        ft1 = FkThread()
        ft1.start()
        # 创建并启动第二个线程
        ft2 = FkThread()
        ft2.start()
print('主线程执行完成!')

In [ ]:
# join的作用
import threading, time
  
class SubThread(threading.Thread):  
    def __init__(self, name):  
        threading.Thread.__init__(self, name=name)  
  
    def run(self):  
        i = 0;  
        while i < 3:  
            print(self.name,'counting...\n')  
            time.sleep(1) 
            print(self.name,'finish\n') 
            i += 1

th = SubThread('thread-1')
print('main start')
th.setDaemon(False)# 可省略,默认为False
th.start()  
th.join() 
print('main end\n')

In [3]:
# 终端运行
# join的作用
# 可以分别取消t1.join() 用于 t2.jion()来查看区别

import time, threading

# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t1 = threading.Thread(target=loop, name='LoopThread1')
t1.start()
t1.join()
t2 = threading.Thread(target=loop, name='LoopThread2')
t2.start()
#t2.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread1 is running...
thread LoopThread1 >>> 1
thread LoopThread1 >>> 2
thread LoopThread1 >>> 3
thread LoopThread1 >>> 4
thread LoopThread1 >>> 5
thread LoopThread1 ended.
thread LoopThread2 is running...
thread LoopThread2 >>> 1
thread MainThread ended.
thread LoopThread2 >>> 2
thread LoopThread2 >>> 3
thread LoopThread2 >>> 4
thread LoopThread2 >>> 5
thread LoopThread2 ended.


### Python sleep函数用法：线程睡眠

如果需要让当前正在执行的线程暂停一段时间，并进入阻塞状态，则可以通过调用 time 模块的 sleep(secs) 函数来实现。该函数可指定一个 secs 参数，用于指定线程阻塞多少秒。
当前线程调用 sleep() 函数进入阻塞状态后，在其睡眠时间段内，该线程不会获得执行的机会，即使系统中没有其他可执行的线程，处于 sleep() 中的线程也不会执行，因此 sleep() 函数常用来暂停程序的运行。

In [4]:
# 当主线程进入睡眠后，系统没有可执行的线程，所以可以看到程序在 sleep() 函数处暂停。
import time

for i in range(10):
    print("当前时间: %s" % time.ctime())
    # 调用sleep()函数让当前线程暂停1s
    time.sleep(1)

当前时间: Tue Jun 18 15:01:19 2019
当前时间: Tue Jun 18 15:01:20 2019
当前时间: Tue Jun 18 15:01:21 2019
当前时间: Tue Jun 18 15:01:22 2019
当前时间: Tue Jun 18 15:01:23 2019
当前时间: Tue Jun 18 15:01:24 2019
当前时间: Tue Jun 18 15:01:25 2019
当前时间: Tue Jun 18 15:01:26 2019
当前时间: Tue Jun 18 15:01:27 2019
当前时间: Tue Jun 18 15:01:28 2019


In [ ]:
# 终端运行
# sleep()

import time, threading

# 新线程执行的代码:
def loop1():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
    print('thread %s ended.' % threading.current_thread().name)

def loop2():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 2:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(3)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t1 = threading.Thread(target=loop2, name='LoopThread1')
t1.start()
t2 = threading.Thread(target=loop1, name='LoopThread2')
t2.start()
t2.join()
print('thread %s ended.' % threading.current_thread().name)

# 线程安全

当使用多个线程来访问同一个数据时，很容易“偶然”出现线程安全问题。

当出现资源竞争时，也会出现线程安全问题

临界资源：
- 临界资源即那些一次只能被一个线程访问的资源，典型例子就是打印机，它一次只能被一个程序用来执行打印功能，因为不能多个线程同时操作，而访问这部分资源的代码通常称之为临界区。

线程安全问题：
关于线程安全，有一个经典的“银行取钱”问题。从银行取钱的基本流程基本上可以分为如下几个步骤：
- 用户输入账户、密码，系统判断用户的账户、密码是否匹配。
- 用户输入取款金额。
- 系统判断账户余额是否大于取款金额。
- 如果余额大于取款金额，则取款成功；如果余额小于取款金额，则取款失败。

使用程序模拟后三步：

Account.py
```python
class Account:
    # 定义构造器
    def __init__(self, account_no, balance):
        # 封装账户编号、账户余额的两个成员变量
        self.account_no = account_no
        self.balance = balance
```
main.py
```python
import threading
import time
import Account

# 定义一个函数来模拟取钱操作
def draw(account, draw_amount):
    # 账户余额大于取钱数目
    if account.balance >= draw_amount:
        # 吐出钞票
        print(threading.current_thread().name\
            + "取钱成功！吐出钞票:" + str(draw_amount))
#        time.sleep(0.001)
        # 修改余额
        account.balance -= draw_amount
        print("\t余额为: " + str(account.balance))
    else:
        print(threading.current_thread().name\
            + "取钱失败！余额不足！")
# 创建一个账户
acct = Account.Account("1234567" , 1000)
# 模拟两个线程对同一个账户取钱
threading.Thread(name='甲', target=draw , args=(acct , 800)).start()
threading.Thread(name='乙', target=draw , args=(acct , 800)).start()

```
多次运行偶尔出现如下结果：
```
甲取钱成功！吐出钞票:800
乙取钱成功！吐出钞票:800
        余额为: 200
        余额为: -600
```

原因：
程序中有两个并发线程在修改 Account 对象，而且系统恰好在注释代码处执行线程切换，切换到另一个修改 Account 对象的线程，所以就出现了问题。

## 同步锁（Lock）
之所以出现上面错误结果，是因为 run() 方法的方法体不具有线程安全性。
为了解决这个问题，Python 的 threading 模块引入了锁（Lock）。threading 模块提供了 Lock 和 RLock 两个类，它们都提供了如下两个方法来加锁和释放锁： 
- acquire(blocking=True, timeout=-1)：请求对 Lock 或 RLock 加锁，其中 timeout 参数指定加锁多少秒。
- release()：释放锁。


Lock 和 RLock 的区别如下：
- threading.Lock：
 - 它是一个基本的锁对象，每次只能锁定一次，其余的锁请求，需等待锁释放后才能获取。

- threading.RLock：
 - 它代表可重入锁（Reentrant Lock）。对于可重入锁，在同一个线程中可以对它进行多次锁定，也可以多次释放。如果使用 RLock，那么 acquire() 和 release() 方法必须成对出现。如果调用了 n 次 acquire() 加锁，则必须调用 n 次 release() 才能释放锁。

- Lock同一时刻只能被上锁一次，而RLock可以被同一所有者上N次锁
- Lock可以被非所有者释放，而RLock只能被所有者释放
> Lock如果出现死锁，所有线程都会卡住。 RLock无此缺点

在实现线程安全的控制中，比较常用的是 RLock。通常使用 RLock 的代码格式如下：

```python
class X:
    #定义需要保证线程安全的方法
    def m () :
        #加锁
        self.lock.acquire()
        try :
            #需要保证线程安全的代码
            ＃...方法体
        #使用finally 块来保证释放锁
        finally :
            #修改完成，释放锁
            self.lock.release()
```

实例：

Account.py
```python
import threading
import time

class Account:
    # 定义构造器
    def __init__(self, account_no, balance):
        # 封装账户编号、账户余额的两个成员变量
        self.account_no = account_no
        self._balance = balance
        self.lock = threading.RLock()

    # 因为账户余额不允许随便修改，所以只为self._balance提供getter方法
    def getBalance(self):
        return self._balance
    # 提供一个线程安全的draw()方法来完成取钱操作
    def draw(self, draw_amount):
        # 加锁
        self.lock.acquire()
        try:
            # 账户余额大于取钱数目
            if self._balance >= draw_amount:
                # 吐出钞票
                print(threading.current_thread().name\
                    + "取钱成功！吐出钞票:" + str(draw_amount))
                time.sleep(0.001)
                # 修改余额
                self._balance -= draw_amount
                print("\t余额为: " + str(self._balance))
            else:
                print(threading.current_thread().name\
                    + "取钱失败！余额不足！")
        finally:
            # 修改完成，释放锁
            self.lock.release()
```
main.py

```python

import threading
import Account

# 定义一个函数来模拟取钱操作
def draw(account, draw_amount):
    # 直接调用account对象的draw()方法来执行取钱操作
    account.draw(draw_amount)
# 创建一个账户
acct = Account.Account("1234567" , 1000)
# 模拟两个线程对同一个账户取钱
threading.Thread(name='甲', target=draw , args=(acct , 800)).start()
threading.Thread(name='乙', target=draw , args=(acct , 800)).start()
```


Lock的用法

In [ ]:
from threading import Thread
some_var = 0
class IncrementThread(Thread):
    def run(self):
        global some_var
        read_value = some_var
        print("some_var in %s is %d" % (self.name, read_value))
        some_var = read_value + 1
        #print "some_var in %s after increment is %d" % (self.name, some_var)
def use_increment_thread():
    threads = []
    for i in range(50):
        t = IncrementThread()
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print("After 50 modifications, some_var should have become 50")
    print("After 50 modifications, some_var is %d" % (some_var,))
use_increment_thread()

In [5]:
# lock
# 可以通过下面两种方式创建一个Lock对象，新创建的 Lock 对象处于未上锁的状态：
import threading
l = threading.Lock()
l

<unlocked _thread.lock object at 0x00000201DB0A0CB0>

In [ ]:
from threading import Lock, Thread
lock = Lock()
some_var = 0
class IncrementThread(Thread):
    def run(self):
        #we want to read a global variable
        #and then increment it
        global some_var
        lock.acquire(True)
        read_value = some_var
        print("some_var in %s is %d" % (self.name, read_value))
        some_var = read_value + 1
        print("some_var in %s after increment is %d" % (self.name, some_var))
        lock.release()
def use_increment_thread():
    threads = []
    for i in range(50):
        t = IncrementThread()
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print("After 50 modifications, some_var should have become 50")
    print("After 50 modifications, some_var is %d" % (some_var,))
use_increment_thread()

In [ ]:
# 不加锁计数器
import time
from threading import Thread
value = 0
def getlock():
    global value
    new = value + 1
    time.sleep(0.001)  # 使用sleep让线程有机会切换
    value = new
threads = []
for i in range(100):
    t = Thread(target=getlock)
    t.start()
    threads.append(t)
for t in threads:
    t.join()
print(value)

In [ ]:
# 加锁保证结果
import time
from threading import Thread, Lock
value = 0
lock = Lock()
def getlock():
    global value
    with lock:
        new = value + 1
        time.sleep(0.001)
        value = new
threads = []
for i in range(100):
    t = Thread(target=getlock)
    t.start()
    threads.append(t)
for t in threads:
    t.join()
print(value)

In [ ]:
import threading  
import time  
  
def test_xc(): 
    mutex.acquire()#取得锁  
    f = open("test.txt","a")  
    f.write("test_dxc"+'\n')  
    f.close()
    mutex.release()#释放锁  

mutex = threading.Lock()#创建锁 

threads = []
for i in range(5):  
    t = threading.Thread(target=test_xc)  
    t.start()  
    threads.append(t)
for t in threads:
    t.join()

## 可重入锁

In [ ]:
import threading

print('lock acquire')
lock = threading.Lock()
lock.acquire()
# 第二次acquire 请求不到，程序会一致卡在这里。
# 因为Lock没有线程从属的概念，所以不能请求2次
lock.acquire()
lock.release()
lock.release()
print('done')

In [ ]:
# RLock 有从属的概念，知道是谁锁的
import threading

print('lock acquire')
lock = threading.RLock()
lock.acquire()
lock.acquire()
lock.release()
lock.release()
print('done')

## Condition

In [ ]:
import threading, time
class Seeker(threading.Thread):
    def __init__(self, cond, name):
        super(Seeker, self).__init__()
        self.cond = cond
        self.name = name
    def run(self):
        time.sleep(1) #确保先运行Seeker中的方法
        self.cond.acquire() #b
        print(self.name + ': 我已经把眼睛蒙上了')
        self.cond.notify()
        self.cond.wait() #c
                         #f
        print(self.name + ': 我找到你了 ~_~')
        self.cond.notify()
        self.cond.release()
                            #g
        print(self.name + ': 我赢了')   #h
class Hider(threading.Thread):
    def __init__(self, cond, name):
        super(Hider, self).__init__()
        self.cond = cond
        self.name = name
    def run(self):
        self.cond.acquire()
        self.cond.wait()    #a    #释放对琐的占用，同时线程挂起在这里，直到被notify并重新占有琐。
                            #d
        print(self.name + ': 我已经藏好了，你快来找我吧')
        self.cond.notify()
        self.cond.wait()    #e
                            #h
        self.cond.release()
        print(self.name + ': 被你找到了，哎~~~')
cond = threading.Condition()
seeker = Seeker(cond, 'seeker')
hider = Hider(cond, 'hider')
seeker.start()
hider.start()

## Event 交通灯

In [ ]:
import threading
import random
import time


class VehicleThread(threading.Thread):
    """Class representing a motor vehicle at an intersection"""

    def __init__(self, threadName, event):
        """Initializes thread"""

        threading.Thread.__init__(self, name=threadName)

        # ensures that each vehicle waits for a green light
        self.threadEvent = event

    def run(self):
        """Vehicle waits unless/until light is green"""

        # stagger arrival times
        time.sleep(random.randrange(1, 10))

        # prints arrival time of car at intersection
        print("%s arrived at %s\n" % \
              (self.getName(), time.ctime(time.time())))

        # wait for green light
        self.threadEvent.wait()

        # displays time that car departs intersection
        print("%s passes through intersection at %s\n" % \
              (self.getName(), time.ctime(time.time())))


greenLight = threading.Event()
vehicleThreads = []

# creates and starts five Vehicle threads
for i in range(1, 5):
    vehicleThreads.append(VehicleThread("Vehicle" + str(i),
                                        greenLight))

for vehicle in vehicleThreads:
    vehicle.start()

while threading.activeCount() > 1:
    # sets the Event's flag to false -- block all incoming vehicles
    greenLight.clear()
    print("RED LIGHT! at", time.ctime(time.time()))
    time.sleep(3)

    # sets the Event's flag to true -- awaken all waiting vehicles
    print("GREEN LIGHT! at", time.ctime(time.time()))
    greenLight.set()
    time.sleep(1)

## 信号量

In [ ]:
import time
from random import random
from threading import Thread, Semaphore
sema = Semaphore(1)
def foo(tid):
    with sema:
        print('{} acquire sema'.format(tid))
        wt = random() * 2
        time.sleep(wt)
    print('{} release sema'.format(tid))
threads = []
for i in range(5):
    t = Thread(target=foo, args=(i,))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

# 进程

In [ ]:
import multiprocessing
import time
def foo(i):
    print('called function in process: %s' % i)
    time.sleep(5)
    return
    
Process_jobs = []
for i in range(5):
    p = multiprocessing.Process(target=foo, args=(i,))
    Process_jobs.append(p)
    p.start()
    
for p in Process_jobs:
    p.join()

## 后台进程

In [ ]:
import multiprocessing
import time

def foo():
    name = multiprocessing.current_process().name
    print ("Starting %s \n" %name)
    time.sleep(3)
    print ("Exiting %s \n" %name)

background_process = multiprocessing.Process\
                    (name='background_process',\
                     target=foo)
background_process.daemon = True
NO_background_process = multiprocessing.Process\
                          (name='NO_background_process',\
                           target=foo)
NO_background_process.daemon = False
background_process.start()
NO_background_process.start()


### 杀死进程

In [ ]:
import multiprocessing
import time
def foo():
    print('Starting function')
    time.sleep(0.1)
    print('Finished function')

p = multiprocessing.Process(target=foo, name='Process-#Test#')
print('Process before execution:', p, p.is_alive())
p.start()
print('Process running:', p, p.is_alive())
p.terminate()
print('Process terminated:', p, p.is_alive())
p.join()
print('Process joined:', p, p.is_alive())
print('Process exit code:', p.exitcode)

## 继承创建进程

In [ ]:
import multiprocessing
class MyProcess(multiprocessing.Process):
    def run(self):
        print('called run method in process: %s' %self.name)
        return


jobs = []
for i in range(5):
    p = MyProcess ()
    jobs.append(p)
    p.start()
    p.join()

# 进程间通信

## 进程共享内存

In [ ]:
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]

if __name__ == '__main__':
    num = Value('d', 0.0)
    arr = Array('i', range(10))

    p = Process(target=f, args=(num, arr))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])

## 进程共享-队列

In [ ]:
# https://docs.python.org/3/library/multiprocessing.html?highlight=queue#multiprocessing.Queue
import multiprocessing
import random
import time
class producer(multiprocessing.Process):
    def __init__(self, queue):
        multiprocessing.Process.__init__(self)
        self.queue = queue
    def run(self) :
        for i in range(10):
            item = random.randint(0, 256)
            self.queue.put(item)
            print("<---Process Producer : item %d appended to queue %s" % (item,self.name))
            time.sleep(1)
            # print("The size of queue is %s" % self.queue.qsize())
class consumer(multiprocessing.Process):
    def __init__(self, queue):
        multiprocessing.Process.__init__(self)
        self.queue = queue
    def run(self):
        time.sleep(3)
        while True:
            # empty or not
            time.sleep(2)
            try:
                item = self.queue.get()
                print('--->Process Consumer : item %d popped from by %s \n' % (item, self.name))
            except Exception as e:
                print("the queue is empty, Process Consumer exit")
                break
        time.sleep(1)

In [ ]:
queue = multiprocessing.Queue()
queue.put(-1) # to avoid queue size (empty, qsize on macos)
process_producer = producer(queue)
process_consumer = consumer(queue)
process_producer.start()
process_consumer.start()
process_producer.join()
process_consumer.join()

# 如果多个consumer，进程同步

### 管道

In [ ]:
from multiprocessing import Process, Pipe
 
class Consumer(Process):
    def __init__(self, pipe):
        Process.__init__(self)
        self.pipe = pipe
 
    def run(self):
        self.pipe.send("Consumer Words")
        print("Consumer Received:", self.pipe.recv())
 
 
class Producer(Process):
    def __init__(self, pipe):
        Process.__init__(self)
        self.pipe = pipe
 
    def run(self):
        print("Producer Received:", self.pipe.recv())
        self.pipe.send("Producer Words")
 
 
pipe = Pipe()
p = Producer(pipe[0])
c = Consumer(pipe[1])
p.daemon = c.daemon = True
p.start()
c.start()
p.join()
c.join()
print("Ended!")

### 进程池

In [ ]:
#阻塞方式
from multiprocessing import Lock, Pool
import time
 
def function(index):
    print("Start process: ", index)
    time.sleep(3)
    print("End process", index)
 
 
pool = Pool(processes=3)
for i in range(4):
    pool.apply(function, (i,))

print("Started processes")
pool.close()
pool.join()
print("Subprocess done.")

In [ ]:
# 非阻塞方式
from multiprocessing import Lock, Pool
import time
 
def function(index):
    print("Start process: ", index)
    time.sleep(3)
    print("End process", index)
 
 
pool = Pool(processes=2)
for i in range(4):
    pool.apply_async(function, (i,))

print("Started processes")
pool.close()
pool.join()
print("Subprocess done.")

# 正则表达式

正则表达式是用于处理字符串的强大工具，拥有自己独特的语法以及一个独立的处理引擎

正则表达式的结构与所创建的算术表达式的结构类似，较大的表达式可由小的表达式通过各种元字符和运算符进行组合而创建

匹配过程：
- 依次拿出表达式和文本中的字符比较
- 如果每一个字符都能匹配，则匹配成功，一旦有匹配不成功的字符则匹配失败

[在线实验](https://regex101.com)

功能：
- 验证：测试字符串内的模式
 - 字符串内是否出现电话号码模式或者信用卡号模式。这称为数据验证
 - 例如手机号，开头三位数运营商，总数11位等
- 查询：基于模式匹配从字符串中提取子字符串
 - 可以查找文档内或输入域内特定的文本
- 替换：替换文本
 - 可以使用正则表达式来识别文档中的特定文本
 - 完全删除该文本或者用其他文本替换它

通过使用内置re模块实现
- 使用python的原始字符串，字符串前面加一个r前缀
- 一般使用步骤：
 - 使用compile函数将正则表达式的字符串形式编译为一个Pattern对象
 - 通过Pattern对象提供的一系列方法对文本进行匹配查找，获得匹配结果
 - 最后使用Match对象提供的属性和方法获得信息，根据需要进行其他操作
 
通用概念
正则表达式包括普通字符和特殊字符(元字符)

元字符：




In [7]:
s = r"<html><body><h1>hello world</h1></body></html>"

In [8]:
# 通过字符串分片解决
start = s.find("<h1>")
end = s.find("</h1>")
print(s[start+4:end])

hello world


In [9]:
import re
help(re)

Help on module re:

NAME
    re - Support for regular expressions (RE).

DESCRIPTION
    This module provides regular expression matching operations similar to
    those found in Perl.  It supports both 8-bit and Unicode strings; both
    the pattern and the strings being processed can contain null bytes and
    characters outside the US ASCII range.
    
    Regular expressions can contain both special and ordinary characters.
    Most ordinary characters, like "A", "a", or "0", are the simplest
    regular expressions; they simply match themselves.  You can
    concatenate ordinary characters, so last matches the string 'last'.
    
    The special characters are:
        "."      Matches any character except a newline.
        "^"      Matches the start of the string.
        "$"      Matches the end of the string or just before the newline at
                 the end of the string.
        "*"      Matches 0 or more (greedy) repetitions of the preceding RE.
                 Greedy 

In [10]:
# 导入库
import re

In [11]:
p1 = r".*<h1>(.*?)</h1>.*"
pattern = re.compile(p1)
groups = re.match(pattern, s)
print(groups.group(1))

hello world


## 查找字符串 Match & Search



In [12]:
name="Hello,My name is tiger,nice to meet you..."
k=re.search(r't(ige)r',name)
if k:
    print(k.group(0),k.group(1))
else:
    print("not search!")


tiger ige


In [13]:
name="Hello,My name is tiger,nice to meet you..."
k=re.match(r"H(....)", name)
if k:
    print(k.group(0),'\n',k.group(1))
else:
    print("not match!")

Hello 
 ello


## 查找所有 FindAll & FindIter

In [14]:
mail='<user01@mail.com> <user02@mail.com> user04@mail.com'
re.findall(r'(\w+@m....[a-z]{3})', mail)

['user01@mail.com', 'user02@mail.com', 'user04@mail.com']

In [15]:
mail_list_iter = re.finditer(r'(\w+@m....[a-z]{3})', mail)

In [16]:
for i in mail_list_iter:
    print(type(i))
    print(i.group())

<class 're.Match'>
user01@mail.com
<class 're.Match'>
user02@mail.com
<class 're.Match'>
user04@mail.com


## 替换

In [17]:
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.



In [18]:
test="Hi, nice to meet you where are you from?"
re.sub(r'\s','-',test)

'Hi,-nice-to-meet-you-where-are-you-from?'

In [19]:
re.sub(r'\s','-',test, 3)

'Hi,-nice-to-meet you where are you from?'

In [20]:
help(re.subn)

Help on function subn in module re:

subn(pattern, repl, string, count=0, flags=0)
    Return a 2-tuple containing (new_string, number).
    new_string is the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in the source
    string by the replacement repl.  number is the number of
    substitutions that were made. repl can be either a string or a
    callable; if a string, backslash escapes in it are processed.
    If it is a callable, it's passed the Match object and must
    return a replacement string to be used.



In [21]:
re.subn(r'\s','-',test, 3)

('Hi,-nice-to-meet you where are you from?', 3)

### 分隔字符串

In [22]:
test="Hi, nice to meet you where are you from?"
re.split(r"\s+",test)

['Hi,', 'nice', 'to', 'meet', 'you', 'where', 'are', 'you', 'from?']

In [23]:
re.split(r"\s+",test,3)

['Hi,', 'nice', 'to', 'meet you where are you from?']